In [ ]:
from pathlib import Path
import sys
import yaml
import time

import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

DIR = Path('..')
sys.path.append(str(DIR))
from gtfo import Gtfo
from gtfo.busSim import BusSim, Config
from gtfo.busSim.manager import managerFactory

%matplotlib inline

In [ ]:
def get_yelp_api_key():
    with open("../config.yml", 'r') as yml:
        cfg = yaml.safe_load(yml)
        return cfg["yelp"]["api_key"]

def plot_background():
    background_path = "../data/plot/background/"
    city = gpd.read_file(background_path + "madison-meter-shp")
    lakes = gpd.read_file(background_path + "water-meter-shp")
    street = gpd.read_file(background_path + "street-meter-shp")
    # city = gpd.read_file(background_path + "madison-shp")
    # lakes = gpd.read_file(background_path + "water-shp")
    # street = gpd.read_file(background_path + "street-shp")

    ax = city.plot(color="lightgray", alpha=.2, figsize=(12, 12), zorder=2)
    lakes.plot(color="lightblue", ax=ax, zorder=1, alpha=0.8)
    street.plot(color="darkgray", alpha = .5, ax=ax, zorder=3)
    return ax

In [ ]:
gtfo = Gtfo("../data/mmt_gtfs.zip")
census_gdf = gtfo.load_census()
config = Config(day="monday", elapse_time="00:30:00", interval="00:10:00", max_walking_min=10)
# config.set_starts(points=[(43.073691, -89.387407), (43.073691, -89.387407)])
config.set_starts(centroids=census_gdf)
t0 = time.time()
result_gdf = gtfo.search(config)
t1 = time.time()
print(t1-t0)

services_gdf = gtfo.load_yelp(api_key=get_yelp_api_key())
gtfo.add_service_metrics(result_gdf, services_gdf)
t2 = time.time()
print(t2-t1)

gtfo.add_demographic_metrics(result_gdf, census_gdf) # SLOW
t3 = time.time()
print(t3-t2)
result_gdf

In [ ]:
result_gdf["banks"].sum() #7263.0

# route remove study

In [27]:
gtfo = Gtfo("../data/mmt_gtfs.zip")
census_gdf = gtfo.load_census()
services_gdf = gtfo.load_yelp(api_key=get_yelp_api_key())

routes = [80,  2, 70,  8, 10,  7,  4, 30,  6, 39, 84, 31,  5, 21, 17, 20, 15, 16, 18, 50, 40, 22, 26, 73, 67, 52, 13, 36, 32]
for route in routes:
    print(route)
    config = Config(day="monday", elapse_time="00:30:00", interval="05:00:00", max_walking_min=10, route_remove=[route])
    config.set_starts(centroids=census_gdf)
    result_gdf = gtfo.search(config)
    gtfo.add_service_metrics(result_gdf, services_gdf)
    gtfo.add_demographic_metrics(result_gdf, census_gdf)
    result_gdf.to_csv(f"../out/result{route}.csv", index=False)

38.18520402908325


# Visualize simulation

In [ ]:
gtfo = Gtfo("../data/mmt_gtfs.zip")
ax = plot_background()
example_gdf = gtfo.load_result_map(map_identifier="search-result-0-600!10")
example_gdf.plot(color="#ffbfba", ax=ax)

# AWS test

In [ ]:
from gtfo.busSim.manager import managerFactory
aws = managerFactory.create("aws", gtfs_path=None, city_path=None, out_path=None)

In [ ]:
aws.clean_up()

In [ ]:
import boto3
aws_lambda = boto3.client('lambda', region_name='ap-northeast-1')
with open("tmp.zip", 'rb') as f:
    response = aws_lambda.create_function(
        Code={
            'ZipFile': f.read()
        },
        Description='BusSim handler',
        FunctionName="test",
        Handler='lambda_function.lambda_handler',
        MemorySize=512,
        Publish=True,
        Role="arn:aws:iam::347664766527:role/s3rwRole",
        Runtime='python3.8',
        Timeout=900,
        TracingConfig={
            'Mode': 'Active',
        },
    )